# Create secrets.yaml

Expected fields (no quotes):

```yaml
client_id: 123
client_secret: abc
client_key: abc
```

# Using the interface:

In [1]:
from tiktok_api_helper.access import get_session

# auto reads from secrets.yaml in the same directory
request_session = get_session()

A query is a combination of a "type (and, or, not)" with multiple Conditions ("Cond")

Each condition is a combination of a "field" (Fields, F), "value" and a operation ("Operations", "Op").

In [2]:
from tiktok_api_helper.Video import TiktokRequest, Query, Cond, Fields, Op

In [3]:
# sample query
req = TiktokRequest(
    query=Query(or_=Cond(Fields.video_id, [123, 456], Op.IN)),
    start_date="20220101",
    end_date="20220129",
).post(request_session)

In [ ]:
hashtags = ["fyp", "foryou", "foryoupage"]

# Multiple conditions
req = TiktokRequest(
    query=Query(
        and_=[
            Cond(Fields.hashtag_name, hashtags, Op.IN),
            Cond(Fields.region_code, "US", Op.EQ),

            # Alternative version with multiple countries - Then the operation changes to "IN" instead of "EQ" (equals) as it's a list
            # the library handles list vs str natively
            # Cond(Fields.region_code, ["US", "UK"], Op.IN),

        ]
    ),
    start_date="20220101",
    end_date="20220129",
).post(request_session)